In [1]:
# from binary_param import linear, constant
from pyomo.environ import (ConcreteModel,
                           SolverFactory,
                           units as pyunits)

import idaes.logger as idaeslog
import pyomo.contrib.parmest.parmest as parmest
import pandas as pd
import pytest

from idaes.core import FlowsheetBlock
from idaes.generic_models.unit_models import Flash
from idaes.generic_models.properties.core.generic.generic_property import (
        GenericParameterBlock)

from idaes.core.util.model_statistics import degrees_of_freedom
import pandas as pd

from idaes.core.util.model_statistics import degrees_of_freedom, large_residuals_set

In [2]:
# binary_params_peng('R32IL298.csv', configuration, 'R32', 'bmimPF6', "x_R32", "x_bmimPF6", 
#     init_temp = 323.15, init_press = 399800, init_x_c1 = 0.5, init_x_c2 = 0.5,
#     init_kappa_2_1 = -0.05, init_kappa_1_2 = -0.03, eps = 0.1, scaling_fac = 1e-9)

In [3]:
def constant(file, configuration, comp_1, comp_2, x_comp_1, x_comp_2,
    init_temp = 323.15, init_press = 399800, init_x_c1 = 0.5, init_x_c2 = 0.5,
    init_kappa_1_2A = -0.11, init_kappa_2_1A = -0.05, eps = 0.0, scaling_fac = 1e-4,read= True):
    """
    Estimates kappa_A parameters for Peng Robinson equation.

    Args:
        file: csv data file in Pa and K
        configuration: imported configuration dictionary
        comp_1: component 1
        comp_2: component 2
        x_comp_1: name of component 1 mole fraction column in csv file
        x_comp_2: name of component 1 mole fraction column in csv file
        init_temp = temperature to initialize model [K]
        init_press = pressure to initialize model [Pa]
        init_x_c1 = component 1 mole fraction to initialize model [mol/mol]
        init_x_c2 = component 1 mole fraction to initialize model [mol/mol]
        init_kappa_A_1_2 = initial guess for kappa_A parameter component 2-component 1
        init_kappa_A_2_1 = initial guess for kappa_A parameter component 1-component 2
        eps = extra
        scaling_fac = 1e-4)
        read: determines if read csv file or not

    Returns:
        printed parameters for binary interaction parameters
    """
    data = file

    def PR_model(data):
        """
        Function which returns initilized model.

        Args:
            data: pandas DataFrame with data

        Returns:
            initialized model
        """
        m = ConcreteModel()

        m.fs = FlowsheetBlock(default={"dynamic": False})

        m.fs.properties = GenericParameterBlock(default=configuration)

        m.fs.state_block = m.fs.properties.state_block_class(
            default={"parameters": m.fs.properties,
                     "defined_state": True})
        x = float(data[x_comp_1])+eps
        m.fs.state_block.flow_mol.fix(1)
        m.fs.state_block.temperature.fix(float(data["temperature"]))
        m.fs.state_block.pressure.fix(float(data["pressure"]))
        m.fs.state_block.mole_frac_comp[comp_2].fix(1-x)
        m.fs.state_block.mole_frac_comp[comp_1].fix(x)

        # parameter - kappa_A_ij (set at 0.3, 0 if i=j)
        m.fs.properties.PR_kappa_A[comp_2, comp_2].fix(0)
        m.fs.properties.PR_kappa_A[comp_2, comp_1].fix(init_kappa_2_1A)
        m.fs.properties.PR_kappa_A[comp_1, comp_1].fix(0)
        m.fs.properties.PR_kappa_A[comp_1, comp_2].fix(init_kappa_1_2A)
        m.fs.properties.PR_kappa_B[comp_2, comp_2].fix(0)
        m.fs.properties.PR_kappa_B[comp_2, comp_1].fix(0)
        m.fs.properties.PR_kappa_B[comp_1, comp_1].fix(0)
        m.fs.properties.PR_kappa_B[comp_1, comp_2].fix(0)
        m.fs.properties.PR_kappa_C[comp_2, comp_2].fix(0)
        m.fs.properties.PR_kappa_C[comp_2, comp_1].fix(0)
        m.fs.properties.PR_kappa_C[comp_1, comp_1].fix(0)
        m.fs.properties.PR_kappa_C[comp_1, comp_2].fix(0)
        # Initialize the flash unit
        m.fs.state_block.initialize(outlvl=idaeslog.CRITICAL)

        # Fix the state variables on the state block
        m.fs.state_block.pressure.fix(float(data["pressure"]))
        m.fs.state_block.temperature.fix(float(data["temperature"]))
        m.fs.state_block.mole_frac_phase_comp['Liq', comp_1].unfix()
        m.fs.state_block.mole_frac_phase_comp['Liq', comp_2].unfix()
        m.fs.state_block.mole_frac_phase_comp['Vap', comp_1].fix(1)
        m.fs.state_block.mole_frac_comp[comp_1].fix(float(data[x_comp_1])+eps)
#         m.fs.state_block.mole_frac_comp[comp_2].fix(float(data[x_comp_2])-eps)
        m.fs.state_block.mole_frac_comp[comp_2].unfix()
        # Set bounds on variables to be estimated
        m.fs.properties.PR_kappa_A[comp_2, comp_1].setlb(-5)
        m.fs.properties.PR_kappa_A[comp_2, comp_1].setub(5)

        m.fs.properties.PR_kappa_A[comp_1, comp_2].setlb(-5)
        m.fs.properties.PR_kappa_A[comp_1, comp_2].setub(5)
        
#         print(degrees_of_freedom(m))

        # Return initialized flash model
        return m

    def SSE(m, data):
        """
        returns objective function expresion.

        Args:
            m: model
            data: pandas DataFrame with data

        Returns:
            objectuve function scaled expresion
        """
        expr = (((float(data[x_comp_2]) - m.fs.state_block.mole_frac_phase_comp['Liq', comp_2]))**2)

        return expr * scaling_fac

    variable_name = ["fs.properties.PR_kappa_A['" + comp_2 + "','" + comp_1 + "']",
                     "fs.properties.PR_kappa_A['" + comp_1 + "','" + comp_2 + "']"]

    pest = parmest.Estimator(PR_model, data, variable_name, SSE, tee=True)

    obj_value, parameters= pest.theta_est(calc_cov=False)

    print_params(obj_value, parameters)
    
    print(large_residuals_set(pest.ef_instance))

#     print("covariance_matrix",a)

def print_params(obj_value, parameters):
    """
    returns printed parameters.

    Args:
        obj_value: objective function value
        parameters: estimated binary parameters

    Returns:
        printed parameters for binary interaction parameters
    """
    print("The SSE at the optimal solution is %0.6f" % obj_value)
    print()
    print("The values for the parameters are as follows:")
    for k,v in parameters.items():
        print(k, "=", v)

In [4]:
data = pd.read_csv('R32IL2.csv')
print(data)
# 283
data_283= data[(data.temperature > 282) & (data.temperature< 284)]

# 298
data_298 = data[(data.temperature > 297) & (data.temperature< 299)]
data_298 = data_298.reset_index(drop=True)

# 323
data_323 = data[(data.temperature > 322) & (data.temperature< 324)]
data_323 = data_323.reset_index(drop=True)

# 323
data_348 = data[(data.temperature > 347) & (data.temperature< 349)]
data_348 = data_348.reset_index(drop=True)

    temperature  pressure  x_R32  x_bmimPF6
0         283.2     99600  0.106      0.894
1         283.2    249700  0.271      0.729
2         283.2    399600  0.428      0.572
3         283.2    549700  0.558      0.442
4         283.2    699600  0.650      0.350
5         283.2    849500  0.815      0.185
6         298.2     99500  0.076      0.924
7         298.2    249700  0.190      0.810
8         298.2    399800  0.297      0.703
9         298.1    549700  0.390      0.610
10        298.1    699500  0.475      0.525
11        298.2    850100  0.552      0.448
12        298.1    999900  0.628      0.372
13        323.1     99500  0.047      0.953
14        323.1    249600  0.113      0.887
15        323.1    399500  0.174      0.826
16        323.1    549700  0.231      0.769
17        323.1    699600  0.285      0.715
18        323.2    849600  0.335      0.665
19        323.2    999600  0.382      0.618
20        348.2     99700  0.029      0.971
21        348.2    249500  0.070

In [5]:
from R32_bmimPF6_PR import configuration

constant(data, configuration, 'R32', 'bmimPF6', "x_R32", "x_bmimPF6",
    init_temp =  283.1, init_press =   399300 , init_x_c1 =    0.448, init_x_c2 = 0.552,
    init_kappa_2_1A = -0.10934254664517969, init_kappa_1_2A = -0.054495915350724095, eps = 0.12, scaling_fac = 1e4)

2021-09-01 18:54:15 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-09-01 18:54:16 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-09-01 18:54:17 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-09-01 18:54:18 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-09-01 18:54:19 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-09-01 18:54:19 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-09-01 18:54:20 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-09-01 18:54:21 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2021-09-01 18:54:22 [INFO] idaes

In [6]:
def linear(file, configuration, comp_1, comp_2, x_comp_1, x_comp_2,
    init_temp = 323.15, init_press = 399800, init_x_c1 = 0.5, init_x_c2 = 0.5,
    init_kappa_1_2A = -0.3, init_kappa_2_1A = 0.5,
    init_kappa_1_2B = -0.3, init_kappa_2_1B = 0.5,
    eps = 0.1, scaling_fac = 1e-9):
    """
    Estimates kappa parameters for Peng Robinson equation.

    Args:
        file: csv data file in Pa and K
        configuration: imported configuration dictionary
        comp_1: component 1
        comp_2: component 2
        x_comp_1: name of component 1 mole fraction column in csv file
        x_comp_2: name of component 1 mole fraction column in csv file
        init_temp = temperature to initialize model [K]
        init_press = pressure to initialize model [Pa]
        init_x_c1 = component 1 mole fraction to initialize model [mol/mol]
        init_x_c2 = component 1 mole fraction to initialize model [mol/mol]
        init_kappa_1_2 = initial guess for kappa parameter component 2-component 1
        init_kappa_2_1 = initial guess for kappa parameter component 1-component 2
        eps = extra
        scaling_fac = 1e-4)

    Returns:
        printed parameters for binary interaction parameters
    """
    data = file

    def PR_model(data):
        """
        Function which returns initilized model.

        Args:
            data: pandas DataFrame with data

        Returns:
            initialized model
        """
        m = ConcreteModel()

        m.fs = FlowsheetBlock(default={"dynamic": False})

        m.fs.properties = GenericParameterBlock(default=configuration)

        m.fs.state_block = m.fs.properties.state_block_class(
            default={"parameters": m.fs.properties,
                     "defined_state": True})
        x = float(data[x_comp_1])+eps
        m.fs.state_block.flow_mol.fix(1)
        m.fs.state_block.temperature.fix(float(data["temperature"]))
        m.fs.state_block.pressure.fix(float(data["pressure"]))
        m.fs.state_block.mole_frac_comp[comp_2].fix(1-x)
        m.fs.state_block.mole_frac_comp[comp_1].fix(x)

        # parameter - kappa_ij (set at 0.3, 0 if i=j)
        m.fs.properties.PR_kappa_A[comp_2, comp_2].fix(0)
        m.fs.properties.PR_kappa_A[comp_2, comp_1].fix(init_kappa_2_1A)
        m.fs.properties.PR_kappa_A[comp_1, comp_1].fix(0)
        m.fs.properties.PR_kappa_A[comp_1, comp_2].fix(init_kappa_1_2A)
        m.fs.properties.PR_kappa_B[comp_2, comp_2].fix(0)
        m.fs.properties.PR_kappa_B[comp_2, comp_1].fix(init_kappa_2_1B)
        m.fs.properties.PR_kappa_B[comp_1, comp_1].fix(0)
        m.fs.properties.PR_kappa_B[comp_1, comp_2].fix(init_kappa_1_2B)
        m.fs.properties.PR_kappa_C[comp_2, comp_2].fix(0)
        m.fs.properties.PR_kappa_C[comp_2, comp_1].fix(0)
        m.fs.properties.PR_kappa_C[comp_1, comp_1].fix(0)
        m.fs.properties.PR_kappa_C[comp_1, comp_2].fix(0)
        # Initialize the flash unit
        m.fs.state_block.initialize(outlvl=idaeslog.CRITICAL)

        # Fix the state variables on the state block
        m.fs.state_block.pressure.fix(float(data["pressure"]))
        m.fs.state_block.temperature.fix(float(data["temperature"]))
        m.fs.state_block.mole_frac_phase_comp['Liq', comp_1].fix(float(data[x_comp_1]))
        m.fs.state_block.mole_frac_phase_comp['Liq', comp_2].fix(float(data[x_comp_2]))
        m.fs.state_block.mole_frac_comp[comp_1].fix(float(data[x_comp_1])+eps)
        m.fs.state_block.mole_frac_comp[comp_2].unfix()

        # Set bounds on variables to be estimated
        m.fs.properties.PR_kappa_A[comp_2, comp_1].setlb(-5)
        m.fs.properties.PR_kappa_A[comp_2, comp_1].setub(5)

        m.fs.properties.PR_kappa_A[comp_1, comp_2].setlb(-5)
        m.fs.properties.PR_kappa_A[comp_1, comp_2].setub(5)

        m.fs.properties.PR_kappa_B[comp_2, comp_1].setlb(-5)
        m.fs.properties.PR_kappa_B[comp_2, comp_1].setub(5)

        m.fs.properties.PR_kappa_B[comp_1, comp_2].setlb(-5)
        m.fs.properties.PR_kappa_B[comp_1, comp_2].setub(5)

        # Return initialized flash model
        return m

    def SSE(m, data):
        """
        returns objective function expresion.

        Args:
            m: model
            data: pandas DataFrame with data

        Returns:
            objectuve function scaled expresion
        """
        expr = ((float(data["pressure"]) - m.fs.state_block.mole_frac_phase_comp['Liq', comp_1])**2)

        return expr * scaling_fac

    variable_name = ["fs.properties.PR_kappa_A['" + comp_1 + "','" + comp_2 + "']",
                     "fs.properties.PR_kappa_A['" + comp_2 + "','" + comp_1 + "']",
                     "fs.properties.PR_kappa_B['" + comp_1 + "','" + comp_2 + "']",
                     "fs.properties.PR_kappa_B['" + comp_2 + "','" + comp_1 + "']"]

    pest = parmest.Estimator(PR_model, data, variable_name, SSE, tee=True)

    obj_value, parameters,a= pest.theta_est(calc_cov=True)

    print_params(obj_value, parameters)

    print("covariance_matrix",a)